In [16]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, median_absolute_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import seaborn as sns
from scipy import stats
import folium
from folium.plugins import HeatMap
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.ticker as ticker

In [17]:
df = pd.read_csv('data/bicing_fin_1.csv')

In [18]:
df.set_index('timestamp_hour', inplace=True)

In [19]:
df.head()

,day_of_week_sin,trend,month_sin,is_weekend,hour_of_day_cos,hour_of_day_sin,lag_168h,seasonality,is_rush_hour,temperature,diff_1h,diff_24h,lag_1h,diff_168h,lag_24h,day_of_week_cos,month_cos,precipitation,log_checkouts,checkouts_hour_station
timestamp_hour,,,,,,,,,,,,,,,,,,,,
2024-07-31 21:00:00,0.974928,NaN,-0.500000,0,0.707107,-0.707107,NaN,175.012496,0,NaN,NaN,NaN,NaN,NaN,NaN,-0.222521,-0.866025,NaN,0.000000,0
2024-07-31 22:00:00,0.974928,NaN,-0.500000,0,0.866025,-0.500000,NaN,175.012496,0,NaN,5.0,NaN,0.0,NaN,NaN,-0.222521,-0.866025,NaN,1.791759,5
2024-07-31 23:00:00,0.974928,NaN,-0.500000,0,0.965926,-0.258819,NaN,175.012496,0,NaN,-4.0,NaN,5.0,NaN,NaN,-0.222521,-0.866025,NaN,0.693147,1
2024-08-01 00:00:00,0.433884,NaN,-0.866025,0,1.000000,0.000000,NaN,367.401068,0,24.0,2.0,NaN,1.0,NaN,NaN,-0.900969,-0.500000,0.0,1.386294,3
2024-08-01 01:00:00,0.433884,NaN,-0.866025,0,0.965926,0.258819,NaN,367.401068,0,24.0,-3.0,NaN,3.0,NaN,NaN,-0.900969,-0.500000,0.0,0.000000,0


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 217202 entries, 2024-07-31 21:00:00 to 2025-01-31 23:00:00
Data columns (total 20 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   day_of_week_sin         217202 non-null  float64
 1   trend                   211259 non-null  float64
 2   month_sin               217202 non-null  float64
 3   is_weekend              217202 non-null  int64  
 4   hour_of_day_cos         217202 non-null  float64
 5   hour_of_day_sin         217202 non-null  float64
 6   lag_168h                208802 non-null  float64
 7   seasonality             217202 non-null  float64
 8   is_rush_hour            217202 non-null  int64  
 9   temperature             217052 non-null  float64
 10  diff_1h                 217152 non-null  float64
 11  diff_24h                216002 non-null  float64
 12  lag_1h                  217152 non-null  float64
 13  diff_168h               208802 non-null  float64

In [21]:
# Apply log1p transformation (log(1+x)) to handle zeros
df['log_checkouts'] = np.log1p(df['checkouts_hour_station'])

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 217202 entries, 2024-07-31 21:00:00 to 2025-01-31 23:00:00
Data columns (total 20 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   day_of_week_sin         217202 non-null  float64
 1   trend                   211259 non-null  float64
 2   month_sin               217202 non-null  float64
 3   is_weekend              217202 non-null  int64  
 4   hour_of_day_cos         217202 non-null  float64
 5   hour_of_day_sin         217202 non-null  float64
 6   lag_168h                208802 non-null  float64
 7   seasonality             217202 non-null  float64
 8   is_rush_hour            217202 non-null  int64  
 9   temperature             217052 non-null  float64
 10  diff_1h                 217152 non-null  float64
 11  diff_24h                216002 non-null  float64
 12  lag_1h                  217152 non-null  float64
 13  diff_168h               208802 non-null  float64

In [26]:
df.head()

,day_of_week_sin,trend,month_sin,is_weekend,hour_of_day_cos,hour_of_day_sin,lag_168h,seasonality,is_rush_hour,temperature,diff_1h,diff_24h,lag_1h,diff_168h,lag_24h,day_of_week_cos,month_cos,precipitation,log_checkouts,checkouts_hour_station
timestamp_hour,,,,,,,,,,,,,,,,,,,,
2024-07-31 21:00:00,0.974928,NaN,-0.500000,0,0.707107,-0.707107,NaN,175.012496,0,NaN,NaN,NaN,NaN,NaN,NaN,-0.222521,-0.866025,NaN,0.000000,0
2024-07-31 22:00:00,0.974928,NaN,-0.500000,0,0.866025,-0.500000,NaN,175.012496,0,NaN,5.0,NaN,0.0,NaN,NaN,-0.222521,-0.866025,NaN,1.791759,5
2024-07-31 23:00:00,0.974928,NaN,-0.500000,0,0.965926,-0.258819,NaN,175.012496,0,NaN,-4.0,NaN,5.0,NaN,NaN,-0.222521,-0.866025,NaN,0.693147,1
2024-08-01 00:00:00,0.433884,NaN,-0.866025,0,1.000000,0.000000,NaN,367.401068,0,24.0,2.0,NaN,1.0,NaN,NaN,-0.900969,-0.500000,0.0,1.386294,3
2024-08-01 01:00:00,0.433884,NaN,-0.866025,0,0.965926,0.258819,NaN,367.401068,0,24.0,-3.0,NaN,3.0,NaN,NaN,-0.900969,-0.500000,0.0,0.000000,0


## **Splitting the dataset into train and test sets**

We'll use a time-based split: 

80% of the data for training (earliest dates)
20% for testing (most recent dates)

In [27]:
df = df.sort_index()

# Determine split points 
n_samples = len(df)
train_size = int(0.8 * n_samples)


In [28]:
# Split the data
train_data = df.iloc[:train_size]
test_data  = df.iloc[train_size:]


In [29]:
print(f"Training data: {len(train_data)} samples")
print(f"Test data: {len(test_data)} samples")

Training data: 173761 samples
Test data: 43441 samples


In [30]:
# Separate features and target
X_train = train_data.drop(['checkouts_hour_station', 'log_checkouts'], axis=1)
y_train = train_data['log_checkouts']

X_test  = test_data.drop(['checkouts_hour_station', 'log_checkouts'], axis=1)
y_test  = test_data['log_checkouts']

In [31]:
X_train.head()

,day_of_week_sin,trend,month_sin,is_weekend,hour_of_day_cos,hour_of_day_sin,lag_168h,seasonality,is_rush_hour,temperature,diff_1h,diff_24h,lag_1h,diff_168h,lag_24h,day_of_week_cos,month_cos,precipitation
timestamp_hour,,,,,,,,,,,,,,,,,,
2024-07-31 21:00:00,0.974928,NaN,-0.5,0,0.707107,-0.707107,NaN,175.012496,0,NaN,NaN,NaN,NaN,NaN,NaN,-0.222521,-0.866025,NaN
2024-07-31 21:00:00,0.974928,NaN,-0.5,0,0.707107,-0.707107,NaN,175.012496,0,NaN,NaN,NaN,NaN,NaN,NaN,-0.222521,-0.866025,NaN
2024-07-31 21:00:00,0.974928,NaN,-0.5,0,0.707107,-0.707107,NaN,175.012496,0,NaN,NaN,NaN,NaN,NaN,NaN,-0.222521,-0.866025,NaN
2024-07-31 21:00:00,0.974928,NaN,-0.5,0,0.707107,-0.707107,NaN,175.012496,0,NaN,NaN,NaN,NaN,NaN,NaN,-0.222521,-0.866025,NaN
2024-07-31 21:00:00,0.974928,NaN,-0.5,0,0.707107,-0.707107,NaN,175.012496,0,NaN,NaN,NaN,NaN,NaN,NaN,-0.222521,-0.866025,NaN


In [32]:
print(f"Train size: {len(X_train)} | Test size: {len(X_test)}")
print(f"Train last datetime: {train_data.index.max()} | Test first datetime: {test_data.index.min()}")

Train size: 173761 | Test size: 43441
Train last datetime: 2024-12-26 07:00:00 | Test first datetime: 2024-12-26 07:00:00


## **Handling Missing Values, Apply One-hot encoding to categorical feature & scaling numerical variables**

In [33]:
# Identify categorical and numerical columns
cat_cols = [col for col in X_train.columns if X_train[col].dtype == 'object' or X_train[col].dtype.name == 'category']
num_cols = [col for col in X_train.columns if col not in cat_cols and X_train[col].dtype != 'bool']
bool_cols = [col for col in X_train.columns if X_train[col].dtype == 'bool']

print(f"Categorical columns: {cat_cols}")
print(f"Numerical columns: {num_cols}")
print(f"Boolean columns: {bool_cols}")

Categorical columns: []
Numerical columns: ['day_of_week_sin', 'trend', 'month_sin', 'is_weekend', 'hour_of_day_cos', 'hour_of_day_sin', 'lag_168h', 'seasonality', 'is_rush_hour', 'temperature', 'diff_1h', 'diff_24h', 'lag_1h', 'diff_168h', 'lag_24h', 'day_of_week_cos', 'month_cos', 'precipitation']
Boolean columns: []


In [34]:
# Handle missing values
from sklearn.impute import SimpleImputer

# Create imputers for numerical features
num_imputer = SimpleImputer(strategy='median')
num_imputer.fit(X_train[num_cols])

SimpleImputer(strategy='median')

In [35]:
# Impute numerical columns
X_train_num_imputed = pd.DataFrame(
    num_imputer.transform(X_train[num_cols]), 
    columns=num_cols, 
    index=X_train.index
)


X_test_num_imputed = pd.DataFrame(
    num_imputer.transform(X_test[num_cols]), 
    columns=num_cols, 
    index=X_test.index
)

In [36]:
# For categorical columns, we use most frequent value
if cat_cols:
    cat_imputer = SimpleImputer(strategy='most_frequent')
    cat_imputer.fit(X_train[cat_cols])
    
    X_train_cat_imputed = pd.DataFrame(
        cat_imputer.transform(X_train[cat_cols]), 
        columns=cat_cols, 
        index=X_train.index
    )
    

    
    X_test_cat_imputed = pd.DataFrame(
        cat_imputer.transform(X_test[cat_cols]), 
        columns=cat_cols, 
        index=X_test.index
    )

In [37]:
# For boolean columns, they don't need imputation, making they stay as-is
if bool_cols:
    X_train_bool = X_train[bool_cols].copy()
    X_test_bool = X_test[bool_cols].copy()

In [38]:
# One-hot encoding for the categorical variables
from sklearn.preprocessing import OneHotEncoder

if cat_cols:
    ohe = OneHotEncoder(sparse_output=False, drop='if_binary')
    ohe.fit(X_train_cat_imputed)
    
    # Get feature names
    cat_feature_names = []
    for i, col in enumerate(cat_cols):
        cat_feature_names.extend([f"{col}_{val}" for val in ohe.categories_[i]])
    
    # Transform data
    X_train_cat_encoded = pd.DataFrame(
        ohe.transform(X_train_cat_imputed), 
        columns=cat_feature_names, 
        index=X_train.index
    )
  
    
    X_test_cat_encoded = pd.DataFrame(
        ohe.transform(X_test_cat_imputed), 
        columns=cat_feature_names, 
        index=X_test.index
    )


In [39]:
# Scale numerical features

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train_num_imputed)

X_train_num_scaled = pd.DataFrame(
    scaler.transform(X_train_num_imputed), 
    columns=num_cols, 
    index=X_train.index
)


X_test_num_scaled = pd.DataFrame(
    scaler.transform(X_test_num_imputed), 
    columns=num_cols, 
    index=X_test.index
)

In [40]:
# Combining the datasets
if cat_cols and bool_cols:
    X_train_processed = pd.concat([X_train_num_scaled, X_train_cat_encoded, X_train_bool], axis=1)
    X_test_processed = pd.concat([X_test_num_scaled, X_test_cat_encoded, X_test_bool], axis=1)
elif cat_cols:
    X_train_processed = pd.concat([X_train_num_scaled, X_train_cat_encoded], axis=1)
    X_test_processed = pd.concat([X_test_num_scaled, X_test_cat_encoded], axis=1)
elif bool_cols:
    X_train_processed = pd.concat([X_train_num_scaled, X_train_bool], axis=1)
    X_test_processed = pd.concat([X_test_num_scaled, X_test_bool], axis=1)
else:
    X_train_processed = X_train_num_scaled
    X_test_processed = X_test_num_scaled

print(f"Final processed training data shape: {X_train_processed.shape}")
print(f"Final processed test data shape: {X_test_processed.shape}")

Final processed training data shape: (173761, 18)
Final processed test data shape: (43441, 18)


## **Building a baseline model**

We'll use a mean predictor as a baseline model to establish a minimal performance benchmark, 

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 217202 entries, 2024-07-31 21:00:00 to 2025-01-31 23:00:00
Data columns (total 20 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   day_of_week_sin         217202 non-null  float64
 1   trend                   211259 non-null  float64
 2   month_sin               217202 non-null  float64
 3   is_weekend              217202 non-null  int64  
 4   hour_of_day_cos         217202 non-null  float64
 5   hour_of_day_sin         217202 non-null  float64
 6   lag_168h                208802 non-null  float64
 7   seasonality             217202 non-null  float64
 8   is_rush_hour            217202 non-null  int64  
 9   temperature             217052 non-null  float64
 10  diff_1h                 217152 non-null  float64
 11  diff_24h                216002 non-null  float64
 12  lag_1h                  217152 non-null  float64
 13  diff_168h               208802 non-null  float64

In [42]:
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Baseline model: predict mean of training log_checkouts
baseline = DummyRegressor(strategy="mean")
baseline.fit(X_train_processed, y_train)

# Predict on test set
y_pred_baseline = baseline.predict(X_test)

# Compute baseline performance on test data
baseline_rmse = np.sqrt(mean_squared_error(y_test, y_pred_baseline))
baseline_mae  = mean_absolute_error(y_test, y_pred_baseline)
baseline_r2   = r2_score(y_test, y_pred_baseline)
print(f"Baseline model (Mean predictor) performance on test set: RMSE={baseline_rmse:.3f}, MAE={baseline_mae:.3f}, R²={baseline_r2:.3f}")

Baseline model (Mean predictor) performance on test set: RMSE=0.686, MAE=0.528, R²=-0.022


## **Machine Learning Regression Models**

We will train several regression models:

- Linear models (Ridge & Lasso)

- Random Forest Regressor

- XGBoost Regressor

First, we need to ensure our modeling process is time-aware. Instead of a standard random cross-validation, we’ll use a rolling time-series cross-validation.

### **Cross-Validation Setup**

We’ll perform 5-fold time series cross-validation on the training set for each model to make sure that the model doesn't see the data from future during the training. 

Creating a dictionnary of models

In [43]:
from sklearn.model_selection import TimeSeriesSplit, cross_validate
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

# Define models with regularization for linear regression.
# Note: The alpha values here are initial choices; you can later tune them.
models = {
    "Ridge": Ridge(alpha=1.0),
    "Lasso": Lasso(alpha=0.1, max_iter=10000),
    "RandomForest": RandomForestRegressor(random_state=42),
    "XGBoost": XGBRegressor(objective="reg:squarederror", random_state=42)
}

# Time series cross-validation
tscv = TimeSeriesSplit(n_splits=5)

# Define scoring metrics for cross-validation
scoring = {
    "rmse": "neg_root_mean_squared_error",
    "mae": "neg_mean_absolute_error",
    "r2": "r2"
}

cv_results = {}
for name, model in models.items():
    scores = cross_validate(model, X_train_processed, y_train, cv=tscv, scoring=scoring, n_jobs=-1)
    mean_rmse = -scores['test_rmse'].mean()
    mean_mae = -scores['test_mae'].mean()
    mean_r2  = scores['test_r2'].mean()
    cv_results[name] = (mean_rmse, mean_mae, mean_r2)
    print(f"{name} CV – RMSE: {mean_rmse:.3f}, MAE: {mean_mae:.3f}, R²: {mean_r2:.3f}")

Ridge CV – RMSE: 0.331, MAE: 0.251, R²: 0.763
Lasso CV – RMSE: 0.364, MAE: 0.272, R²: 0.728
RandomForest CV – RMSE: 0.031, MAE: 0.004, R²: 0.997
XGBoost CV – RMSE: 0.054, MAE: 0.025, R²: 0.987


In [45]:
for name, model in models.items():
    model.fit(X_train_processed, y_train)
    y_pred = model.predict(X_test_processed)

    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print(f"{name} Test – RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")

/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Ridge Test – RMSE: 0.454, MAE: 0.205, R²: 0.552


/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Lasso Test – RMSE: 0.404, MAE: 0.270, R²: 0.645


/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


RandomForest Test – RMSE: 0.031, MAE: 0.001, R²: 0.998
XGBoost Test – RMSE: 0.026, MAE: 0.011, R²: 0.999


/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


**Checking Predictions in Real Units**

In [46]:
y_test_actual = np.expm1(y_test)
y_pred_actual = np.expm1(y_pred)


In [48]:
from sklearn.metrics import mean_absolute_error, r2_score, root_mean_squared_error
rmse = root_mean_squared_error(y_test_actual, y_pred_actual)
mae = mean_absolute_error(y_test_actual, y_pred_actual)
r2 = r2_score(y_test_actual, y_pred_actual)

print(f"RMSE (real units): {rmse:.2f} bikes")
print(f"MAE (real units): {mae:.2f} bikes")
print(f"R² (real units): {r2:.3f}")

RMSE (real units): 0.16 bikes
MAE (real units): 0.06 bikes
R² (real units): 0.998



On average, the model's prediction is off by only ~0.16 bikes per hour.

The average absolute difference between predicted and actual is just 0.06 bikes.

Our model explains 99.8% of the variance in hourly checkouts.


In [51]:
best_model = None
best_model_name = ""
lowest_rmse = float("inf")

In [52]:
for name, model in models.items():
    model.fit(X_train_processed, y_train)
    y_pred_log = model.predict(X_test_processed)

    # Evaluate in log space
    rmse_log = mean_squared_error(y_test, y_pred_log, squared=False)

    # Transform to real units
    y_test_actual = np.expm1(y_test)
    y_pred_actual = np.expm1(y_pred_log)

    # Evaluate in real units
    rmse = root_mean_squared_error(y_test_actual, y_pred_actual)
    mae = mean_absolute_error(y_test_actual, y_pred_actual)
    r2 = r2_score(y_test_actual, y_pred_actual)

    print(f"{name} Test – RMSE: {rmse:.2f} bikes, MAE: {mae:.2f}, R²: {r2:.3f}")

    # Check if this model is the best so far
    if rmse < lowest_rmse:
        lowest_rmse = rmse
        best_model = model
        best_model_name = name

/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Ridge Test – RMSE: 53144631828456259584.00 bikes, MAE: 254985794032747680.00, R²: -234030315292036595914100462119381106688.000


/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Lasso Test – RMSE: 4205523687043.57 bikes, MAE: 20177631039.72, R²: -1465525836064075510972416.000


/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


RandomForest Test – RMSE: 0.24 bikes, MAE: 0.00, R²: 0.995
XGBoost Test – RMSE: 0.16 bikes, MAE: 0.06, R²: 0.998


/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


## **Saving the Best Model**

In [55]:
print(f"Best model: {best_model_name} with RMSE = {lowest_rmse:.2f} bikes")


Best model: XGBoost with RMSE = 0.16 bikes


In [58]:
# Save the best model
import joblib

joblib.dump(best_model, "models/final_model.joblib")


['models/final_model.joblib']